In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [21]:
import requests

# 대기오염정보 조회
url_base = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc'
url_spec = 'getMinuDustFrcstDspth'
url = url_base + '/' + url_spec
api_key_utf8 = 'D5v8NPs87kaeUioo2TfiZzJaPeE4mwxyzbDv9UXl1gRRvudDM2I3%2B1Pj1PV2Luz9ZG5NRlub6UYoN%2F5a%2BCuPLQ%3D%3D'
api_key_decode = requests.utils.unquote(api_key_utf8, encoding='utf-8')

# 오늘 날짜로 검색
search_date = '2024-04-04',
params = {
    'serviceKey' : api_key_decode,
    'returnType' : 'xml',
    'searchDate' : search_date,
    'ver' : '1.1'
}
response = requests.get(url, params=params)

In [22]:
print(response)

<Response [200]>


In [23]:
print(response.text)

<?xml version="1.0" encoding="UTF-8"?>
<response>
  <header>
    <resultCode>00</resultCode>
    <resultMsg>NORMAL_CODE</resultMsg>
  </header>
  <body>
    <items>
      <item>
        <actionKnack/>
        <informOverall>○ [미세먼지] 전 권역이 &apos;좋음&apos;∼&apos;보통&apos;으로 예상됩니다.</informOverall>
        <informGrade>서울 : 보통,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋음,경북 : 보통,울산 : 좋음,대구 : 보통,부산 : 좋음,충남 : 좋음,충북 : 좋음,세종 : 좋음,대전 : 좋음,영동 : 보통,영서 : 좋음,경기남부 : 보통,경기북부 : 좋음,인천 : 보통</informGrade>
        <imageUrl3>https://www.airkorea.or.kr/file/proxyImage?fileName=2024/04/04/AQF.20240403.NIER_09_01.PM10.1hsp.2024040415.png</imageUrl3>
        <imageUrl2>https://www.airkorea.or.kr/file/proxyImage?fileName=2024/04/04/AQF.20240403.NIER_09_01.PM10.1hsp.2024040409.png</imageUrl2>
        <imageUrl1>https://www.airkorea.or.kr/file/proxyImage?fileName=2024/04/04/AQF.20240403.NIER_09_01.PM10.1hsp.2024040403.png</imageUrl1>
        <informCode>PM10</informCode>
        <imageUrl4>https://www.airkorea.or.kr/fi

In [24]:
print(response.content.decode(encoding='utf-8')) # response의 내용을 디코딩하여 문자열로 변환하고 출력하는 작업을 수행

<?xml version="1.0" encoding="UTF-8"?>
<response>
  <header>
    <resultCode>00</resultCode>
    <resultMsg>NORMAL_CODE</resultMsg>
  </header>
  <body>
    <items>
      <item>
        <actionKnack/>
        <informOverall>○ [미세먼지] 전 권역이 &apos;좋음&apos;∼&apos;보통&apos;으로 예상됩니다.</informOverall>
        <informGrade>서울 : 보통,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋음,경북 : 보통,울산 : 좋음,대구 : 보통,부산 : 좋음,충남 : 좋음,충북 : 좋음,세종 : 좋음,대전 : 좋음,영동 : 보통,영서 : 좋음,경기남부 : 보통,경기북부 : 좋음,인천 : 보통</informGrade>
        <imageUrl3>https://www.airkorea.or.kr/file/proxyImage?fileName=2024/04/04/AQF.20240403.NIER_09_01.PM10.1hsp.2024040415.png</imageUrl3>
        <imageUrl2>https://www.airkorea.or.kr/file/proxyImage?fileName=2024/04/04/AQF.20240403.NIER_09_01.PM10.1hsp.2024040409.png</imageUrl2>
        <imageUrl1>https://www.airkorea.or.kr/file/proxyImage?fileName=2024/04/04/AQF.20240403.NIER_09_01.PM10.1hsp.2024040403.png</imageUrl1>
        <informCode>PM10</informCode>
        <imageUrl4>https://www.airkorea.or.kr/fi

In [25]:
# beautifulsoup을 통해 xml 형식의 문자열을 파싱 (파서는 lxml 사용)
from bs4 import BeautifulSoup

xml = BeautifulSoup(response.text, 'lxml')

In [26]:
type(xml)

bs4.BeautifulSoup

In [27]:
xml.find('header')

<header>
<resultcode>00</resultcode>
<resultmsg>NORMAL_CODE</resultmsg>
</header>

In [28]:
# 필요한 라이브러리 선언
import pandas as pd

# 문자열을 처리하는 함수
def convert_string(item_, key_):
    try:
        return item.find(key_.lower()).text.strip() # _key를 소문자로 변환하여 검색하고 찾은 값에 양쪽의 공백을 제거한 후 반환
    except AttributeError: # item에서 key_를 찾을 수 없는 경우 AttributeError 발생
        return None
    
items = xml.findAll('item') # xml 문서에서 모든 item 요소 찾기
item_list = [] #item 요소들을 저장할 목적의 빈 리스트 생성

for item in items:
    # item 요소에서 필요한 정보를 추출해 해당 정보의 키와 값을 가진 딕셔너리 item_dict를 만듦
    # convert_string() 함수를 호출하여 각 요소에서 필요한 정보를 추출하고, 해당 키에 맞게 딕셔너리에 저장
    item_dict = {
        '통보 시간': convert_string(item, 'dataTime'),
        '통보 코드' : convert_string(item, 'informCode'),
        '예보 개황' : convert_string(item, 'infromOverall'),
        '발생 원인' : convert_string(item, 'informCause'),
        '예보 등급' : convert_string(item, 'informGrade'),
        '행동 요령' : convert_string(item, 'actionKnack'),
        '첨부 파일명1': convert_string(item, 'imageUrl1'),
        '첨부 파일명2': convert_string(item, 'imageUrl2'),
        '첨부 파일명3': convert_string(item, 'imageUrl3'),
        '첨부 파일명4': convert_string(item, 'imageUrl4'),
        '첨부 파일명5': convert_string(item, 'imageUrl5'),
        '첨부 파일명6': convert_string(item, 'imageUrl6'),
        '첨부 파일명7': convert_string(item, 'imageUrl7'),
        '첨부 파일명8': convert_string(item, 'imageUrl8'),
        '첨부 파일명9': convert_string(item, 'imageUrl9'),
        '예측 통보 시간': convert_string(item, 'informData')
    }
    item_list.append(item_dict) # 만들어진 item_dict를 item_list에 추가
df = pd.DataFrame(item_list) # item_list에 저장된 dict을 이용하여 데이터 프레임 생성

df.to_excel(excel_writer="./air_info.xlsx") # 데이터 프레임을 엑셀 파일로 저장

In [29]:
df.iloc[0]

통보 시간                                       2024-04-04 11시 발표
통보 코드                                                    PM10
예보 개황                                                    None
발생 원인            ○ [미세먼지] 원활한 대기 확산으로 대기질이 대체로 청정할 것으로 예상됩니다.
예보 등급       서울 : 보통,제주 : 좋음,전남 : 좋음,전북 : 좋음,광주 : 좋음,경남 : 좋...
행동 요령                                                        
첨부 파일명1     https://www.airkorea.or.kr/file/proxyImage?fil...
첨부 파일명2     https://www.airkorea.or.kr/file/proxyImage?fil...
첨부 파일명3     https://www.airkorea.or.kr/file/proxyImage?fil...
첨부 파일명4     https://www.airkorea.or.kr/file/proxyImage?fil...
첨부 파일명5     https://www.airkorea.or.kr/file/proxyImage?fil...
첨부 파일명6     https://www.airkorea.or.kr/file/proxyImage?fil...
첨부 파일명7     https://www.airkorea.or.kr/file/proxyImage?fil...
첨부 파일명8     https://www.airkorea.or.kr/file/proxyImage?fil...
첨부 파일명9     https://www.airkorea.or.kr/file/proxyImage?fil...
예측 통보 시간                                           2024-04-04
Name: 0,